# 3 Quantum Simulations

In this section, we aim at using the knowledge above with some extention to do simple simlulation problems.

## Tutorial 1: Introduction to Qiskit patterns

I will not try to copy everything under this section, as they contain too much link to the external sources. I will look at those I read, and may mention them in the following sections, like when we use it in section 4.

There are many partners for Qiskit, and they are divided into four sections:

## 1.1 Map the problem to quantum circuits and operators

The "map" step is the foundational phase of a Qiskit pattern where users transform classical problems into quantum-native representations—typically quantum circuits or operators. This translation process is critical for applications ranging from chemistry simulations to quantum optimization algorithms.

## Core Purpose and Outputs

This step involves encoding the problem into abstract circuits or operators that represent quantum states and observables. For instance, chemistry problems require constructing quantum circuits representing molecular wave faunctions, with operators defining the energy Hamiltonian. The output is typically a collection of quantum circuits or operators ready for optimization.

## Tools for Circuit Construction

### Circuit Library
Qiskit's circuit library provides an extensive collection of standard gates and higher-level quantum operations, including:
- **Pauli gates**: X, Y, Z
- **Rotation gates**: RX, RY, RZ
- **Controlled operations**: CNOT, Toffoli

These gates serve as building blocks for constructing quantum algorithms. The library also includes specialized circuits like the QFT and variational ansätze commonly used in VQE applications.

### Operators and SparsePauliOp
The `SparsePauliOp` class is the primary tool for representing quantum operators as linear combinations of Pauli strings. This is particularly crucial for specifying observables in quantum chemistry and simulation tasks. 

`SparsePauliOp` enables efficient operator arithmetic for hundreds of qubits when the number of non-zero Pauli terms is sufficiently small, making it memory-efficient compared to dense matrix representations. Users can initialize operators using the `from_sparse_list` method, specifying Pauli strings, qubit indices, and coefficients.

## Advanced Capabilities

### Classical Feedforward and Control Flow
Dynamic circuits enable mid-circuit measurements with classical feedforward operations, allowing quantum gate execution to depend on measurement outcomes. Qiskit implements this through the `if_test()` method, which creates conditional execution blocks. 

This capability is essential for algorithms requiring:
- Adaptive measurements
- Error correction protocols
- Efficient state preparation

**Note**: Mid-circuit measurements typically have longer execution times than two-qubit gates, potentially offsetting benefits from reduced circuit depth.

### OpenQASM 3 Integration
OpenQASM 3 serves as an intermediate representation language supporting classical feedforward and control flow based on measurement outcomes. It enables explicit timing specifications and low-level pulse definitions for calibration tasks. 

Qiskit provides bidirectional conversion tools:
- `qasm3.dump()` - Export circuits to OpenQASM strings
- `qasm3.loads()` - Import OpenQASM programs into QuantumCircuit objects


## 1.2 Optimizing for Target Hardware

The "optimize for target hardware" step transforms abstract quantum circuits from the mapping phase into executable Instruction Set Architecture (ISA) circuits that can run on physical quantum hardware. This critical optimization process balances circuit fidelity, execution time, and resource constraints to maximize the likelihood of successful quantum computations.

## Core Purpose and ISA Circuits

This step performs a series of optimizations including routing circuits to physical qubit layouts, converting gates to hardware-native basis gates, and reducing operation counts. Abstract circuits must be transpiled to ISA circuits—circuits containing only gates understood by the target hardware (basis gates) and respecting connectivity constraints (coupling map). As of March 2024, only ISA circuits can be executed on IBM hardware using Qiskit Runtime.

## The Qiskit Transpiler

### Transpilation Stages

The transpiler operates through distinct stages, each addressing specific optimization challenges:

**1. Layout Stage**: Maps logical circuit qubits to physical hardware qubits. The layout significantly impacts subsequent routing efficiency and overall circuit performance.

**2. Routing Stage**: Inserts SWAP gates to satisfy hardware connectivity constraints. The routing algorithm ensures all multi-qubit gates operate on adjacent physical qubits according to the device's coupling map.

**3. Translation Stage**: Rewrites all gates into the hardware's native basis gates. For instance, if hardware only supports CZ gates but the circuit requests CX gates, the translation stage decomposes CX into CZ and single-qubit rotations.

**4. Optimization Stage**: Reduces gate count and circuit depth through various optimization passes. This includes canceling redundant gates, commuting operations, and consolidating consecutive single-qubit gates.

**5. Scheduling Stage** (optional): Inserts explicit delay instructions to control timing, which is essential for implementing dynamical decoupling and other time-sensitive error suppression techniques.

### Optimization Levels

Qiskit provides four optimization levels (0-3) offering different trade-offs between compilation speed and circuit quality:

- **Level 0**: Minimal optimization—performs basic layout and routing with no gate optimization
- **Level 1**: Light optimization with basic gate cancellation
- **Level 2**: Default setting providing balanced optimization suitable for most use cases
- **Level 3**: Heavy optimization employing sophisticated algorithms to minimize gate count and depth, though significantly slower for large circuits

Higher optimization levels typically produce circuits with fewer two-qubit gates and reduced depth, directly improving fidelity since multi-qubit gates have higher error rates and qubits decohere over time.

## Advanced Optimization Tools

### Qiskit Transpiler Service and AI-Powered Passes

The Qiskit Transpiler Service leverages IBM Cloud resources and AI-powered transpiler passes to achieve superior circuit optimization. Available to Premium Plan users, this service provides:

**AI Routing Pass**: Uses reinforcement learning algorithms to intelligently select layouts and perform circuit routing. The AI models typically outperform traditional heuristic algorithms, particularly for large circuits.

**AI Synthesis Passes**: Employ machine learning to optimize specific circuit blocks:
- **AICliffordSynthesis**: Optimizes Clifford circuits (H, S, CX gates) up to 9 qubits
- **AILinearFunctionSynthesis**: Optimizes linear function circuits (CX, SWAP gates) up to 9 qubits  
- **AIPermutationSynthesis**: Optimizes permutation circuits for up to 65 qubits

The AI-powered passes can reduce two-qubit gate counts by an average of 42% when combined with heuristic passes. They operate both locally (with additional dependencies) or remotely via the cloud service. Users can enable AI optimization by setting `ai="true"` when instantiating the transpiler service, or use `ai="auto"` to let the service intelligently decide based on circuit characteristics.

### Pass Managers

Pass managers orchestrate the transpilation process by composing individual transformation passes:

**Preset Pass Managers**: The `generate_preset_pass_manager` function creates pre-configured pass managers based on optimization level and target backend, providing a convenient high-level interface.

**Custom Pass Managers**: Users can create custom pass managers by composing individual passes for specialized optimization workflows. This enables fine-grained control over the transpilation pipeline.

**Staged Pass Managers**: Organize transpilation into distinct stages (pre-layout, layout, routing, translation, optimization, scheduling), with each stage defined by its own pass manager. Stages can be customized or replaced independently.

## Error Suppression: Dynamical Decoupling

Dynamical decoupling is a crucial error suppression technique that inserts sequences of gates (like X-X or XY4 sequences) into idle periods of quantum circuits. These gate sequences compose to the identity but cancel unwanted environmental interactions, reducing decoherence effects.

Implementation requires:
1. **Scheduling Analysis**: Using `ALAPScheduleAnalysis` or `ASAPScheduleAnalysis` to add timing information to circuits
2. **PadDynamicalDecoupling Pass**: Inserting the DD sequences into identified idle windows with proper spacing

Dynamical decoupling works best when circuits have significant idle time—situations where some qubits remain inactive while others execute gates. The technique requires careful consideration of:
- Gate duration and timing constraints
- The specific DD sequence chosen (X-X, XY4, Uhrig, etc.)
- Pulse alignment requirements for the target hardware
- Mid-circuit measurement latency for dynamic circuits

## Debugging and Testing Tools

### Exact Simulation

**Qiskit SDK Primitives**: Reference implementations provide exact simulation for small circuits, useful for verifying circuit correctness before hardware execution.

**Qiskit Aer**: High-performance simulation framework supporting:
- **Multiple simulation methods**: Statevector, density matrix, unitary, matrix product state, stabilizer
- **Noise models**: Both automatic noise models based on real device calibration data and custom noise models for research
- **GPU acceleration**: Support for CUDA-enabled GPUs to accelerate simulations of circuits with 20+ qubits
- **Large-scale simulation**: Stabilizer circuits can be efficiently simulated with thousands of qubits

### Local Testing Mode

Qiskit Runtime's local testing mode allows users to test primitives (Sampler and Estimator) with simulators before submitting to quantum hardware. Users can employ:
- **AerSimulator**: Flexible high-performance simulator with configurable noise models
- **Fake Backends**: Simulators that mimic specific IBM QPUs using calibration snapshots, automatically applying realistic noise models

### Qiskit Runtime Neat Class

For analyzing and debugging Estimator jobs specifically, the Neat class provides tools to examine job performance, identify issues, and optimize execution parameters.

## Qiskit Functions

The Qiskit Functions Catalog offers Premium Plan users access to optimized execution workflows:

**Circuit Functions**: Provide simplified interfaces receiving abstract circuits and observables, then managing synthesis, optimization, and execution automatically. These include:
- **IBM Circuit Function**: Incorporates AI-powered transpilation extensions and advanced error mitigation
- **Q-CTRL Performance Management**: AI-driven quantum control techniques for scaling to larger problems
- **QEDMA QESEM**: Efficient characterization-based error suppression and mitigation
- **Algorithmiq TEM**: Tensor-network error mitigation requiring fewer shots than traditional methods

Circuit functions bring together cutting-edge transpilation, error suppression, and error mitigation to deliver utility-grade performance out of the box, allowing researchers to focus on problem mapping rather than optimization details.

## Key Practices

1. **Choose appropriate optimization levels**: Use level 0 for rapid prototyping, level 1-2 for production workflows, and level 3 only when circuit quality is paramount and compilation time is acceptable

2. **Leverage AI passes strategically**: Enable AI-powered optimization for circuits where traditional heuristics struggle, particularly large circuits with complex connectivity requirements

3. **Implement dynamical decoupling**: Add DD sequences to circuits with substantial idle time to suppress decoherence errors

4. **Test with simulators first**: Use local testing mode and Aer simulators to validate circuits before consuming quantum hardware time

5. **Consider circuit structure early**: Design circuits that approximately follow device topology during the mapping phase to facilitate more efficient optimization

6. **Monitor transpilation metrics**: Track gate counts, circuit depth, and estimated error rates to assess optimization effectiveness

This optimization step is critical for bridging the gap between abstract quantum algorithms and physical quantum hardware execution, directly impacting the quality and reliability of quantum computations in the utility era.


## 1.3 Executing on Hardware

The "execute on hardware" step runs optimized ISA circuits on quantum processing units (QPUs) and produces quantum computation outputs. This phase bridges the gap between prepared circuits and actionable results, utilizing Qiskit Runtime primitives with configurable error suppression and mitigation techniques to maximize result quality.

## Qiskit Runtime Primitives

Primitives provide a high-level abstraction for the two fundamental quantum computing tasks: sampling quantum states and computing expectation values. They handle low-level execution details, allowing users to focus on algorithm development rather than hardware management.

### Sampler Primitive

**Purpose**: Samples output registers from quantum circuit execution, producing measurement outcome distributions.

**Use Cases**:
- Quantum sampling algorithms
- Circuit verification and debugging  
- Generating bitstring distributions for optimization problems
- Probabilistic computations

**Output**: Per-shot measurements as bitstrings with associated counts. For example, a 3-qubit circuit might return `{'000': 412, '111': 612}` indicating measurement frequencies across 1024 shots.

**Key Features**:
- Does not perform error mitigation (use mthree package for local mitigation)
- Supports error suppression techniques (dynamical decoupling, gate twirling)
- Returns raw sampling data without post-processing

### Estimator Primitive

**Purpose**: Computes expectation values of observables with respect to quantum states prepared by circuits.

**Use Cases**:
- Variational quantum eigensolvers (VQE)
- Quantum chemistry simulations
- Optimization algorithms (QAOA)
- Cost function evaluation in hybrid quantum-classical workflows

**Output**: Expectation values of observables corresponding to physical quantities or cost functions. For instance, computing the ground state energy of a molecule represented as a Hamiltonian observable.

**Key Features**:
- Supports comprehensive error mitigation (TREX, ZNE, PEC, PEA)
- Provides resilience levels for configurable cost/accuracy trade-offs
- Returns expectation values with optional confidence intervals

### Primitive Unified Blocs (PUBs)

Circuits and parameters are packaged as PUBs—tuples containing:
1. A quantum circuit (possibly parameterized)
2. Observable(s) to measure (Estimator only)  
3. Parameter values for circuit instantiation

PUBs support array-valued observables and parameter values with broadcasting, enabling efficient batch processing of related computations.

## Execution Modes

Qiskit Runtime offers three execution modes optimizing job scheduling and resource utilization based on workload characteristics.

### Job Mode

**Description**: Single primitive request without a context manager, executing one computational unit.

**When to Use**: 
- Single job submissions
- Simple experiments not requiring multiple iterations
- Quick prototyping and testing

**Initialization**: `mode=backend` when instantiating primitives

**Characteristics**:
- Simplest execution pattern
- Each job enters the standard queue independently
- No special scheduling optimizations

### Batch Mode

**Description**: Multi-job manager for efficiently running experiments with independent, non-conditional jobs.

**When to Use**:
- Multiple independent circuits to execute simultaneously
- Comparing different error mitigation configurations
- Parameter sweeps without inter-job dependencies
- When you have all inputs ready at the outset

**Initialization**: `mode=batch` or use batch context manager

**Key Benefits**:
- Parallel classical processing (up to 5 jobs)
- Jobs scheduled together for efficiency
- More cost-effective than sessions
- Individual job results available for selective resubmission
- Only QPU time counts as usage (not compilation time)

**Characteristics**:
- First job enters normal queue
- Subsequent jobs run with reduced queuing delays
- Can be interrupted by calibrations or software upgrades
- Interactive TTL reactivates workload if jobs arrive within timeout

### Session Mode

**Description**: Dedicated execution window providing exclusive QPU access for iterative workloads.

**When to Use**:
- Variational algorithms (VQE, QAOA) requiring classical optimization loops
- Iterative workloads with classical post-processing between quantum jobs
- Experiments needing tightly coupled execution
- When you need dedicated hardware access

**Initialization**: `mode=session` or use session context manager

**Key Benefits**:
- Exclusive QPU access during active window
- No queuing delays for jobs within the session
- Ideal for algorithms sensitive to device drift
- Can learn and reuse noise models across jobs (with PEA/PEC)
- Supports parallel job submission

**Characteristics**:
- More expensive than batch mode (usage = wall clock time from first job until session closes)
- Open Plan users cannot use sessions
- Maximum and interactive TTL timers control session lifecycle
- Thread-safe for parallel workload submission
- Never interrupted by calibrations

**Best Practices**:
- Close sessions promptly to avoid unnecessary usage accumulation
- For jobs consuming >1 minute QPU time, consider breaking into smaller parallel jobs
- Combine with optimization to reduce total wall-clock time

## Error Suppression and Mitigation

Qiskit Runtime provides sophisticated techniques to combat quantum noise and improve result accuracy.

### Error Suppression Techniques

**Dynamical Decoupling (DD)**: Inserts pulse sequences (X-X, XY4, Uhrig) on idle qubits to cancel environmental interactions, reducing decoherence. Automatically applied at optimization level 1+ for Estimator.

**Gate Twirling**: Randomly compiles gates into equivalent sequences that average out coherent errors, converting them into stochastic noise more amenable to mitigation.

**Measurement Twirling**: Applies randomization to measurement bases, helping suppress measurement errors and readout bias.

These techniques modify circuits or pulses to reduce errors at their source, requiring minimal overhead.

### Error Mitigation Techniques

Error mitigation processes outputs from circuit ensembles to reduce errors (bias) in results, typically requiring computational overhead:

**TREX (Twirled Readout Error eXtinction)**: Model-free technique combining measurement twirling with readout error mitigation. Applied at resilience level 1.

**ZNE (Zero Noise Extrapolation)**: Amplifies circuit noise through digital gate folding (e.g., U → UU†U), runs circuits at multiple noise levels, then extrapolates to the zero-noise limit. Does not guarantee unbiased results but often substantially improves accuracy. Available at resilience level 2.

**PEA (Probabilistic Error Amplification)**: Variant of ZNE that uses specific noise amplification strategies tailored for improved performance.

**PEC (Probabilistic Error Cancellation)**: Most powerful mitigation technique providing unbiased expectation value estimates. Learns comprehensive noise model and inverts errors through probabilistic sampling. Significant overhead in model learning and circuit sampling. Available at resilience level 3 (Estimator V1 only).

### Resilience Levels

Estimator provides four resilience levels trading accuracy for computational cost:

- **Level 0**: No error mitigation
- **Level 1**: TREX (measurement mitigation and twirling)
- **Level 2**: Level 1 + ZNE
- **Level 3**: Most comprehensive mitigation with PEC (V1 only)

Higher levels produce more accurate results but require longer processing times and increased sampling overhead.

### Custom Error Settings

Users can individually enable/disable specific techniques:

```python
estimator = Estimator(backend)
estimator.options.resilience.measure_mitigation = True
estimator.options.resilience.zne_mitigation = True
estimator.options.dynamical_decoupling.enable = True
estimator.options.twirling.enable_gates = True
```

**Important Considerations**:
- Not all techniques work together on all circuit types (see compatibility tables)
- Sampler does not support error mitigation natively (use mthree package instead)
- Error mitigation increases computational overhead—choose techniques appropriate to your accuracy requirements

## Job Management and Monitoring

### Job Lifecycle

1. **Submission**: Jobs enter queue upon submission
2. **Queuing**: Jobs wait for QPU availability according to fair-share scheduling
3. **Execution**: QPU processes the job, applying configured error handling
4. **Post-processing**: Classical processing applies error mitigation and formats results
5. **Completion**: Results become available via `job.result()`

### Monitoring Jobs

**Status Checking**: 
```python
print(f"Job ID: {job.job_id()}")
print(f"Status: {job.status()}")
```

**Progress Tracking**: Monitor job progress through the IBM Quantum Platform dashboard or programmatically via API

**Usage Metrics**: Track QPU time consumption:
```python
qpu_time = job.metrics()["usage"]["quantum_seconds"]
```

### Job Limits and Optimization

**Maximum Execution Time**: Each execution mode has maximum TTL limits. Sessions have both maximum TTL (total session duration) and interactive TTL (time between jobs).

**Minimize Runtime**:
- Combine small jobs (< 1 minute QPU time) into larger jobs to amortize fixed overhead
- Use batch mode for independent jobs to leverage parallel classical processing
- Break large jobs into smaller ones for session parallelism

**Fair-Share Scheduling**: IBM Quantum uses fair-share algorithms distributing QPU time proportionally across users and projects within each access plan.

**Job Tags**: Organize and filter jobs using custom tags for better experiment management.

## Qiskit Functions

The Qiskit Functions Catalog (Premium Plan users) provides managed execution workflows combining transpilation, error handling, and execution:

**Circuit Functions**: Simplified interfaces accepting abstract circuits and observables, automatically managing the full execution pipeline:

- **IBM Circuit Function**: AI-powered transpilation with advanced error mitigation (ZNE via PEA at mitigation level 3)
- **Q-CTRL Performance Management**: AI-driven quantum control for scaling to larger problems
- **QEDMA QESEM**: Efficient characterization-based error suppression and mitigation
- **Algorithmiq TEM**: Tensor-network error mitigation requiring fewer shots than PEC

These functions deliver utility-grade performance out-of-the-box, abstracting away optimization details so researchers can focus on algorithm development.

## Platform Information

### Processor Types

IBM Quantum offers various processor architectures with different qubit counts, connectivity patterns, and performance characteristics. Users should select backends based on:
- Circuit requirements (qubit count, connectivity)
- Calibration data (error rates, coherence times)
- Availability and queue times

### Backend Selection

**Least Busy**: `service.least_busy(operational=True, simulator=False, min_num_qubits=127)`

**Specific Backend**: `backend = service.backend('ibm_brisbane')`

**Calibration Data**: Access real-time calibration information including gate fidelities, T1/T2 coherence times, and readout errors through backend properties.

### Dynamic Repetition Rate

Advanced feature allowing circuits to be executed at variable repetition rates based on circuit characteristics, optimizing QPU utilization.

### Cost Management

**Workload Usage**: Monitor consumption through the dashboard or programmatically. Usage accumulates differently for each execution mode (job: QPU time; batch: sum of QPU times; session: wall clock time during active window).

**Plan Limits**: Open Plan users have monthly limits; Premium and Flex Plan users have different cost structures. Always close sessions promptly and optimize job sizing to control costs.

## Key Practices

1. **Choose the right primitive**: Sampler for distributions, Estimator for expectation values
2. **Select appropriate execution mode**: Job for single runs, batch for independent jobs, session for iterative workflows
3. **Configure error handling strategically**: Balance accuracy needs against computational overhead
4. **Optimize job sizes**: Combine small jobs, parallelize within sessions
5. **Monitor and manage resources**: Track usage, use job tags, close sessions promptly
6. **Consider Qiskit Functions**: Leverage managed services for complex workflows
7. **Test with simulators first**: Validate circuits before consuming QPU time

This execution step transforms optimized quantum circuits into actionable results, bridging the gap between quantum hardware capabilities and practical computational needs in the quantum utility era.


## 1.4 Post-process Results

While basic post-processing typically involves saving and visualizing computational results, Sample-based Quantum Diagonalization (SQD) represents a sophisticated classical post-processing technique that operates on measurement samples obtained from quantum circuits executed on quantum processing units (QPUs). This hybrid approach bridges quantum and distributed classical computing to extract eigenvalues and eigenvectors of quantum operators—particularly Hamiltonians of quantum systems.

For researchers simulating chemical systems or investigating quantum phenomena (such as molecular ground states, excited states, or quantum phase transitions), SQD offers a powerful framework to refine noisy quantum measurements into accurate spectral information. This methodology is especially relevant for quantum chemistry applications, materials science, and quantum simulation studies where understanding the energy landscape and electronic structure is paramount.

---

## Overview of SQD Methodology

### Theoretical Foundation

The Sample-based Quantum Diagonalization technique addresses a fundamental challenge in quantum computing: extracting meaningful eigenstate information from noisy, finite-sample quantum measurements. The method operates on the interacting-electron Hamiltonian:

$$\hat{H} = \sum_{\substack{pr\\ \sigma}}h_{pr} \hat{a}_{p\sigma}^\dagger \hat{a}_{r\sigma} + \sum_{\substack{prqs\\ \sigma\tau}} \frac{1}{2}\left(pr|qs\right)\hat{a}_{p\sigma}^\dagger \hat{a}_{q\tau}^\dagger \hat{a}_{s\tau}\hat{a}_{r\sigma}$$

where:
- $\hat{a}_{p\sigma}^\dagger$ and $\hat{a}_{p\sigma}$ are fermionic creation and annihilation operators
- $h_{pr}$ represents one-body electronic integrals
- $(pr|qs)$ represents two-body electron repulsion integrals
- $p, q, r, s$ index spatial orbitals, and $\sigma, \tau$ denote spin states

### Key Advantages

1. **Noise Resilience**: SQD can extract accurate ground state information even from highly noisy quantum measurements
2. **Scalability**: The technique naturally parallelizes across distributed computing resources
3. **Flexibility**: Works with various ansatz circuits and quantum hardware backends
4. **Self-Consistency**: Iterative refinement improves accuracy through configuration recovery

---

## Workflow Implementation: N₂ Molecule Case Study

### 1. Molecular System Preparation

The workflow begins with defining the quantum system—in this case, the nitrogen molecule (N₂) at equilibrium geometry. The molecular Hamiltonian requires three key components:

```python
from pyscf import ao2mo, tools

# System parameters
num_orbitals = 16        # Number of spatial orbitals
num_alpha = num_beta = 5 # Electrons per spin channel
open_shell = False       # Closed-shell configuration
spin_sq = 0             # Total spin quantum number

# Load molecular data from electronic structure calculations
molecule_scf = tools.fcidump.to_scf("n2_fci.txt")

# Extract Hamiltonian components
core_hamiltonian = molecule_scf.get_hcore()  # One-body terms
electron_repulsion_integrals = ao2mo.restore(1, molecule_scf._eri, num_orbitals)
nuclear_repulsion_energy = molecule_scf.mol.energy_nuc()
```

**Critical Insight**: These electronic integrals encode the complete many-body physics of the system. The one-body terms capture kinetic energy and electron-nuclear interactions, while two-body terms represent electron-electron repulsions—the computationally challenging part of quantum chemistry.

### 2. Quantum Sampling Phase

In a production workflow, this phase involves:
1. Constructing a parameterized quantum ansatz circuit
2. Optimizing circuit parameters (e.g., via variational quantum eigensolver)
3. Transpiling to hardware-native gates
4. Executing on quantum hardware using the Sampler primitive

```python
# Production workflow (commented for demonstration)
# from qiskit_ibm_runtime import SamplerV2 as Sampler
# sampler = Sampler(mode=backend)
# job = sampler.run([isa_circuit], shots=10_000)
# counts = job.result()[0].data.meas.get_counts()

# For demonstration: generate synthetic measurement data
from qiskit_addon_sqd.counts import generate_counts_uniform, counts_to_arrays

rng = np.random.default_rng(24)
counts = generate_counts_uniform(10_000, num_orbitals * 2, rand_seed=rng)

# Convert raw counts to structured arrays
bitstring_matrix_full, probs_array_full = counts_to_arrays(counts)
```

**Note**: The synthetic data generation simulates a 32-qubit quantum circuit (16 orbitals × 2 spins) measured with 10,000 shots. In practice, these measurements would contain both quantum and classical noise from the QPU.

### 3. Self-Consistent Configuration Recovery Loop

This is the heart of the SQD algorithm—an iterative refinement process that progressively improves the quality of sampled quantum states.

#### Configuration Parameters

```python
# SQD iteration control
ITERATIONS = 5            # Number of refinement cycles

# Subspace solver configuration
NUM_BATCHES = 1          # Independent diagonalization batches
SAMPLES_PER_BATCH = 500  # Samples per eigenstate calculation
MAX_DAVIDSON_CYCLES = 200 # Davidson solver iterations
```

#### Iterative Refinement Algorithm

The configuration recovery loop implements a three-stage process:

**Stage 1: Configuration Recovery** (Iterations 2-N)
```python
if avg_occupancy is None:
    # First iteration: use raw measurements
    bitstring_matrix_tmp = bitstring_matrix_full
    probs_array_tmp = probs_array_full
else:
    # Subsequent iterations: refine based on learned occupancies
    bitstring_matrix_tmp, probs_array_tmp = recover_configurations(
        bitstring_matrix_full,
        probs_array_full,
        avg_occupancy,  # Prior knowledge from previous iteration
        num_alpha,
        num_beta,
        rand_seed=rng
    )
```

**Key Mechanism**: The `recover_configurations()` function probabilistically flips bits in the measurement bitstrings based on the average orbital occupancy learned from previous iterations. This effectively "repairs" noise-corrupted measurements by biasing them toward physically reasonable configurations.

**Stage 2: Post-selection and Subsampling**
```python
batches = postselect_and_subsample(
    bitstring_matrix_tmp,
    probs_array_tmp,
    hamming_right=num_alpha,  # Correct number of spin-up electrons
    hamming_left=num_beta,    # Correct number of spin-down electrons
    samples_per_batch=SAMPLES_PER_BATCH,
    num_batches=NUM_BATCHES,
    rand_seed=rng
)
```

**Physical Constraint**: This step enforces particle number conservation—a fundamental symmetry of the fermion system. Any measurement violating the correct electron count is discarded as unphysical.

**Stage 3: Subspace Diagonalization**
```python
for j in range(NUM_BATCHES):
    strs_a, strs_b = bitstring_matrix_to_ci_strs(batches[j])
    
    # Solve fermion problem in the sampled configuration subspace
    energy_sci, coeffs_sci, avg_occs, spin = solve_fermion(
        batches[j],
        core_hamiltonian,
        electron_repulsion_integrals,
        open_shell=open_shell,
        spin_sq=spin_sq,
        max_cycle=MAX_DAVIDSON_CYCLES
    )
    
    # Add nuclear repulsion to electronic energy
    energy_sci += nuclear_repulsion_energy
    
    # Aggregate results
    e_tmp[j] = energy_sci
    s_tmp[j] = spin
    occs_tmp.append(avg_occs)

# Compute average orbital occupancy across all batches
avg_occupancy = tuple(np.mean(occs_tmp, axis=0))
```

**Computational Insight**: The `solve_fermion()` function performs exact diagonalization within the selected configuration subspace using the Davidson iterative eigensolver. This converts the exponentially large Hilbert space problem into a tractable subspace calculation, with the subspace intelligently chosen based on quantum measurements.

### 4. Convergence Analysis and Visualization

The iterative process tracks three key metrics:
- **Ground state energy**: Convergence to exact eigenvalue
- **Spin expectation value**: Verification of quantum numbers
- **Orbital occupancy**: Electronic structure characterization

```python
# Define convergence benchmark
chem_accuracy = 0.001  # 1 mH ≈ 1 kcal/mol (chemical accuracy)
exact_energy = -109.04667  # Full CI reference energy

# Extract convergence data
energy_error = [abs(np.min(energy_hist[i]) - exact_energy) 
                for i in range(ITERATIONS)]

# Visualize energy convergence
plt.plot(range(ITERATIONS), energy_error, marker='o')
plt.axhline(y=chem_accuracy, linestyle='--', 
            label='Chemical accuracy', color='#BF5700')
plt.yscale('log')
plt.ylabel('Energy Error (Ha)')
plt.xlabel('SQD Iteration')
```

**Performance Analysis**: The demonstration shows convergence to ~27 mH error after 5 iterations—approaching chemical accuracy despite starting from pure noise. This remarkable noise tolerance stems from the self-consistent refinement leveraging physical constraints (particle number, spin symmetry) and learned electronic structure patterns.

---

## Post-Processing Insights and Best Practices

### 1. Parallelization Strategy

The batching mechanism (`NUM_BATCHES`) enables distributed computing:

```python
# Conceptual parallelization (would use actual parallel framework)
from concurrent.futures import ProcessPoolExecutor

def solve_batch(batch_data):
    return solve_fermion(batch_data, core_hamiltonian, 
                        electron_repulsion_integrals, ...)

with ProcessPoolExecutor() as executor:
    results = list(executor.map(solve_batch, batches))
```

**Scaling Consideration**: For large molecular systems (>50 orbitals), distributing batch calculations across multiple compute nodes becomes essential to maintain tractable wall-clock times.

### 2. Convergence Diagnostics

Monitor multiple convergence indicators:

- **Energy stagnation**: When $\Delta E < 10^{-6}$ Ha between iterations
- **Occupancy stability**: When $|\mathbf{n}_{i+1} - \mathbf{n}_i| < 10^{-3}$
- **Spin consistency**: $\langle \hat{S}^2 \rangle$ remains at target value

```python
# Example convergence check
def check_convergence(energy_hist, occupancy_hist, threshold=1e-6):
    if len(energy_hist) < 2:
        return False
    
    energy_change = abs(energy_hist[-1] - energy_hist[-2])
    occ_change = np.linalg.norm(
        np.array(occupancy_hist[-1]) - np.array(occupancy_hist[-2])
    )
    
    return (energy_change < threshold) and (occ_change < 1e-3)
```

### 3. Orbital Occupancy Interpretation

The final occupancy profile reveals electronic structure:

```python
# Combine spin-up and spin-down occupancies
total_occupancy = occupancy_hist[-1][0] + occupancy_hist[-1][1]

# Visualize orbital character
plt.bar(range(len(total_occupancy)), total_occupancy)
plt.axhline(y=2.0, linestyle='--', color='gray', label='Full occupancy')
plt.xlabel('Spatial Orbital Index')
plt.ylabel('Average Occupancy')
```

**Chemical Insight**: For N₂, strong occupancy in the first 5 orbitals (approaching 2.0) corresponds to the σ and π bonding molecular orbitals. This matches chemical intuition about triple-bonded nitrogen.

### 4. Error Analysis and Uncertainty Quantification

Assess result reliability through:

```python
# Standard error across batches
energy_std = np.std(energy_hist[-1])
energy_sem = energy_std / np.sqrt(NUM_BATCHES)

print(f"Final energy: {np.mean(energy_hist[-1]):.6f} ± {energy_sem:.6f} Ha")

# Bootstrap confidence intervals
from scipy import stats
confidence_interval = stats.t.interval(
    0.95, 
    NUM_BATCHES - 1,
    loc=np.mean(energy_hist[-1]),
    scale=energy_sem
)
```

---

## Integration with Broader Workflows

### Connection to Variational Methods

SQD complements variational quantum eigensolvers (VQE):

1. **VQE Phase**: Optimize circuit parameters to minimize energy
2. **SQD Phase**: Post-process VQE samples to refine ground state

This two-stage approach can outperform either method alone, especially in the presence of noise.

### Application to Excited States

Extend SQD to excited state calculations:

```python
# Target multiple eigenstates
NUM_STATES = 3

energy_spectrum, state_vectors = solve_fermion(
    batches[j],
    core_hamiltonian,
    electron_repulsion_integrals,
    nroots=NUM_STATES,  # Request multiple roots
    ...
)
```

**Use Case**: Optical absorption spectra, photochemistry, or quantum dynamics simulations.

### Integration with Digital Twin Frameworks

For researchers working with digital twin methodologies (particularly in quantum system simulation):

- **Model Validation**: SQD provides ground truth benchmarks for reduced-order quantum models
- **Uncertainty Quantification**: Statistical analysis of batch results informs model confidence bounds
- **Real-time Monitoring**: Convergence metrics serve as digital twin synchronization signals

---

## Computational Considerations

### Memory Requirements

Dominant memory consumers:
1. **Bitstring storage**: $O(N_{\text{samples}} \times N_{\text{qubits}})$
2. **CI coefficient vectors**: $O(N_{\text{configs}}^2)$ for Davidson solver
3. **ERI tensor**: $O(N_{\text{orbitals}}^4)$ for Hamiltonian matrix elements

**Optimization**: Use sparse matrix representations and memory-mapped arrays for large systems.

### Time Complexity

- Configuration recovery: $O(N_{\text{samples}} \times N_{\text{orbitals}})$
- Subspace diagonalization: $O(N_{\text{davidson}} \times N_{\text{configs}}^2)$
- Total per iteration: Dominated by eigensolve, scales as $O(N_{\text{configs}}^3)$ worst case

**Practical Runtime**: For the N₂ example, ~10 seconds per iteration on a modern workstation. For larger molecules (e.g., Fe-porphyrin with ~100 orbitals), expect minutes to hours per iteration.

---

## Practices Summary

1. **Start with quality samples**: Better ansatz → fewer SQD iterations needed
2. **Tune batch parameters**: Balance $N_{\text{batches}}$ vs. $N_{\text{samples/batch}}$ for your computational resources
3. **Monitor spin and symmetry**: Use built-in quantum number checks to catch errors early
4. **Save intermediate results**: Store occupancies and energies at each iteration for diagnostics
5. **Leverage parallelism**: Distribute batch calculations across available cores/nodes

---

## Conclusion

Sample-based Quantum Diagonalization exemplifies the power of hybrid quantum-classical algorithms. By intelligently post-processing quantum measurements through iterative refinement and physics-informed constraints, SQD extracts accurate spectral information that would otherwise be obscured by quantum noise. This methodology is invaluable for near-term quantum applications in chemistry, materials science, and quantum simulation—domains where understanding energy landscapes is crucial.

The N₂ case study demonstrates that even with entirely synthetic noisy data, SQD converges to near-chemical-accuracy ground state energies within a handful of iterations. For researchers integrating quantum computing into their workflows, SQD provides a robust, well-tested framework for extracting maximum value from quantum hardware measurements.


# Tutorial 2: Hello World- Qiskit Patterns notebook

This notebook is a external source for a simple application of what we read about from above.

## Overview
Introduction to the **Qiskit Patterns** workflow using Sampler and Estimator primitives. Creates a Bell state and demonstrates the four-step pattern: Map → Optimize → Execute → Post-process.

---

## Core Workflow Structure

### Step 1: Map the Problem
Define quantum circuits and observables for measurement.

**Observable Definition (Estimator)**
```python
observables_labels = ["IZ", "IX", "ZI", "XI", "ZZ", "XX"]
observables = [SparsePauliOp(label) for label in observables_labels]
```
- Bell state expectation: ZZ ≈ 1, XX ≈ 1 (entanglement signature)
- Single-qubit observables: IZ, IX, ZI, XI ≈ 0

**Measurement Gates (Sampler)**
```python
qc.measure_all()  # Adds barrier + measurements + classical register 'meas'
```

---

### Step 2: Optimize for Hardware

**Transpilation to ISA Circuit**
```python
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

backend = FakeAlmadenV2()  # Can use FakeValenciaV2, FakeTorino, FakeBrisbane
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(qc)
```

Key points:
- ISA = Instruction Set Architecture
- Converts to backend's native gate set
- Handles qubit connectivity constraints
- Optimization levels: 0 (none) to 3 (maximum)

---

### Step 3: Execute Using Primitives

**Estimator Primitive**
```python
from qiskit_ibm_runtime import EstimatorV2 as Estimator

estimator = Estimator(backend)
mapped_observables = [obs.apply_layout(isa_circuit.layout) for obs in observables]
job = estimator.run([(isa_circuit, mapped_observables)])
result = job.result()
```

**Sampler Primitive**
```python
from qiskit_ibm_runtime import SamplerV2 as Sampler

sampler = Sampler(backend)
job = sampler.run([isa_circuit], shots=1024)
result = job.result()
```

---

### Step 4: Post-Process Results

**Estimator: Expectation Values**
```python
pub_result = result[0]
values = pub_result.data.evs  # Expectation values array
```

**Sampler: Probability Distribution**
```python
counts = result[0].data.meas.get_counts()
# Note: 'meas' is default name from measure_all()
```

---

## Real Device Execution

```python
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(channel="ibm_quantum", instance="YOUR_CRN")
backend = service.least_busy(operational=True, simulator=False)

# Same workflow as simulator, just different backend
```

**Job Retrieval**
```python
service = QiskitRuntimeService()
retrieved_job = service.job('JOB_ID')
result = retrieved_job.result()
```

---

No need to spend too much time on this, as it is only for steps illustration, not really aiming at the content.

# Tutorial 3: Lecture Notes for Quantum Simulation

This is quite important for knowledge, although not related to coding.